In [1]:
from typing import Callable, Dict, Optional, Tuple, Union, Any, collections
import torch
import copy 
import random
import numpy as np
import sys
import time

import spacy
from spacy.lang.en import English
from result_statistics import result_statistics


from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
import torch.nn.functional as F

from torch.utils.data import Dataset

from transformers.file_utils import (
    WEIGHTS_NAME,
    is_apex_available,
    is_datasets_available,
    is_in_notebook,
    is_sagemaker_distributed_available,
    is_torch_tpu_available,
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    SequenceClassifierOutputWithPast,
)

from transformers import GPT2LMHeadModel, GPT2Model

import os
import time
import datetime
import torch
import math
import copy 
import random
from packaging import version
import pandas as pd
import numpy as np
import pickle

from torch import nn
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, GPT2Model
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainerState, TrainingArguments

from transformers.utils import logging
logger = logging.get_logger(__name__)

In [2]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [3]:
batch_size = 4
MAX_LEN = 128
CAN_NUM = 20
gen_sent_len = 30
num_sent_gen = 3
readable_context = False


# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

SAVE_PATH = "/mnt/nfs/work1/llcao/zonghaiyao/LM/"

global debug
debug = {}

In [4]:
class wiki2021_GPT2Dataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids
    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [5]:
class rerankGPT2LMHeadModel_generation_sent(GPT2LMHeadModel):
    def __init__(self, config, MAX_LEN=None, CAN_NUM=None, num_of_rerank=None):
        super().__init__(config)
        self.MAX_LEN = MAX_LEN
        self.CAN_NUM = CAN_NUM
        self.num_of_rerank = num_of_rerank
        self.VOCAB_SIZE = config.vocab_size
        
        self.transformer = GPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.rerank_linear_head = nn.Linear(config.n_embd, 1, bias=False)

        self.init_weights()

In [6]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|endoftext|>') #gpt2-medium
tokenizer_GPT2 = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|endoftext|>') #gpt2-medium


model = rerankGPT2LMHeadModel_generation_sent.from_pretrained(SAVE_PATH + "results/baseline_wiki2021/exclude_cases_label_not_in_candidates_canNUM20/120000",
                                                                                    config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

rerankGPT2LMHeadModel_generation_sent(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        

In [7]:
# with open(SAVE_PATH + 'data/wiki2021/wiki2021_0to4_train_dataset.pkl', 'rb') as f:
#     train_input_ids = pickle.load(f)
# with open(SAVE_PATH + 'data/wiki2021/wiki2021_0to4_validation_dataset.pkl', 'rb') as f:
#     validation_input_ids = pickle.load(f)
with open(SAVE_PATH + 'data/wiki2021/wiki2021_0to4_inside_validation_dataset.pkl', 'rb') as f:
    inside_validation_input_ids = pickle.load(f)
# train_dataset = wiki2021_GPT2Dataset(train_input_ids)
# validation_dataset = wiki2021_GPT2Dataset(validation_input_ids)
inside_validation_dataset = wiki2021_GPT2Dataset(inside_validation_input_ids)

# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
# train_dataloader = DataLoader(
#             train_dataset,  # The training samples.
#             sampler = RandomSampler(train_dataset), # Select batches randomly
#             batch_size = batch_size # Trains with this batch size.
#         )

# For validation the order doesn't matter, so we'll just read them sequentially.
# validation_dataloader = DataLoader(
#             validation_dataset, # The validation samples.
#             sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )

# For inside_validation the order doesn't matter, so we'll just read them sequentially.
inside_validation_dataloader = DataLoader(
            inside_validation_dataset, # The validation samples.
            sampler = SequentialSampler(inside_validation_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [8]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

def top_k_logits(logits, k, filling_value=-1e10):
    #modified from https://github.com/graykode/gpt-2-Pytorch/blob/master/GPT2/sample.py
    if k == 0:
        return logits
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1].view(-1, 1).expand_as(logits)
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * filling_value, logits)

def print_sampled_sent(generated_sent, outf, print_prefix):
    outf.write(print_prefix + ': ' + generated_sent + '\n')

global debug
debug = {}
def sample_seq_stage1(model, context, gen_sent_len, device, temperature=1, top_k = 20, sample=True):
#def sample_seq(model_condition, context, insert_loc, future_emb_chosen_arr, gen_sent_len, device, temperature=1, top_k = 5, sample=True):
    #modified from https://github.com/graykode/gpt-2-Pytorch/blob/master/GPT2/sample.py
    global debug
    prev = context
    batch_size = prev.size(0)
    output = torch.zeros((batch_size, 0), dtype=torch.long, device = device)
    past = None
    #sample = False
    for i in range(gen_sent_len):
        outputs = model.transformer(prev, past_key_values = past)
        logits = model.lm_head(outputs.last_hidden_state)
        past = outputs.past_key_values
        logits = logits[:, -1, :] / temperature
        logits = top_k_logits(logits, k=top_k)
        
        probs = F.softmax(logits, dim=-1)

        if sample:
            if torch.isnan(probs).sum() > 0:
                print(past)
                print(prev)
                print(insert_loc)
                print(future_emb_chosen_arr)
                print(logits)
                print(probs)
                sys.exit(0)
            prev = torch.multinomial(probs, num_samples=1)
        else:
            _, prev = torch.topk(probs, k=1, dim=-1)
        output = torch.cat((output, prev), dim=1)
    return output

def sample_seq_stage2(model, context, gen_sent_len, device, temperature=1, top_k = 20, sample=True):
#def sample_seq(model_condition, context, insert_loc, future_emb_chosen_arr, gen_sent_len, device, temperature=1, top_k = 5, sample=True):
    #modified from https://github.com/graykode/gpt-2-Pytorch/blob/master/GPT2/sample.py
    global debug
    prev = context
    batch_size = prev.size(0)
    output = torch.zeros((batch_size, 0), dtype=torch.long, device = device)
    past = None
    #sample = False
    for i in range(gen_sent_len):
        debug['prev'] = prev
        debug['past'] = past
        outputs = model.transformer(prev, past_key_values = past)
        logits = model.lm_head(outputs.last_hidden_state)[:, -1, :]
        past = outputs.past_key_values
        
        candidate_token_logits, candidate_token_ids = torch.topk(logits, top_k)
        #make context for rerank stage, 50256 is the token_id for </endoftext/>
        sep_token = torch.ones(size = [candidate_token_ids.shape[0], 1], dtype = torch.long, device=prev.device) * 50256
        candidate_context_ids = torch.cat([sep_token, candidate_token_ids, sep_token, candidate_token_ids], -1)
        rerank_outputs = model.transformer(candidate_context_ids,
                                          past_key_values=past
                                          )
        rerank_logits = model.rerank_linear_head(rerank_outputs.last_hidden_state[:, 2+top_k:2+top_k*2])  
        probs = F.softmax(rerank_logits, dim=1).reshape([-1, top_k])
        if sample:
            if torch.isnan(probs).sum() > 0:
                print(past)
                print(prev)
                print(insert_loc)
                print(future_emb_chosen_arr)
                print(logits)
                print(probs)
                sys.exit(0)
            
            tmp_prev = torch.multinomial(probs, num_samples=1).tolist()
            prev = []
            for i in range(probs.shape[0]):
                prev.append([candidate_token_ids[i][tmp_prev[i][0]].tolist()])
            prev = torch.tensor(prev, device=probs.device)
#         else:
#             _, prev = torch.topk(probs, k=1, dim=-1)
        output = torch.cat((output, prev), dim=1)
    return output

In [9]:
# model = model.to(device)

# t1 = time.time()

# model.eval()

# # Evaluate data for one epoch
# for batch in inside_validation_dataloader:   
#     batch = batch.to(device)
#     with torch.no_grad():  
        
#         output = sample_seq_stage2(model=model,
#                                    context=batch[:, :20], 
#                                    gen_sent_len=30,
#                                    device=device,
#                                    temperature=1,
#                                    top_k = 20, 
#                                    sample=True)



In [10]:
def preprocessing_context(context, outf = None, shortest_context = 50, longest_context = 150):
    bad_context = False
    context = context.replace('â',"'").replace('â','-').replace('\n'," ")
    if len(context.split()) < shortest_context:
    #if len(context.split()) < 5:
        if outf is not None:
            outf.write("Skip due to short context\n")
        bad_context = True
    #if len(context.split()) > 50:
    if len(context.split()) > longest_context:
        if outf is not None:
            outf.write("Skip due to long context\n")
        bad_context = True
    try:
        context.encode('ascii', 'strict')
    except:
        if outf is not None:
            outf.write("Skip due to special token\n")
        bad_context = True
    return context, bad_context

def get_word_list_spacy(inner_idx_tensor, feature_text, tokenizer_GPT2, nlp):
    def get_word_list_from_text(feature_text_i):
        feature_text_i_str = tokenizer_GPT2.convert_tokens_to_string(feature_text_i)
        tokens = nlp.tokenizer(feature_text_i_str)
        word_raw_list_i_j = []

        for tok in tokens:
            w = tok.text
            word_raw_list_i_j.append(w)
            
        return word_raw_list_i_j
    word_raw_list = []
    word_raw_rest_list = []
    batch_size, num_head = inner_idx_tensor.size()
    inner_idx_tensor_np = inner_idx_tensor.cpu().numpy()
    for b, feature_text_i in enumerate(feature_text):
        word_raw_list_i = []
        word_raw_rest_list_i = []
        for j in range(num_head):
            end_idx = inner_idx_tensor_np[b,j]
            word_raw_list_i_j = get_word_list_from_text(feature_text_i[:end_idx])
            #assert len(word_idx_list_i_j) > 0, print(feature_text_i[:end_idx])
            word_raw_list_i.append(word_raw_list_i_j)
            if end_idx == len(feature_text_i):
                word_raw_rest_list_i.append([])
            else:
                word_raw_rest_list_i_j = get_word_list_from_text(feature_text_i[end_idx:])
                word_raw_rest_list_i.append(word_raw_rest_list_i_j)
            #count = word_idx_d2_count.get(w_idx,0)
            #word_idx_d2_count[w_idx] += 1
        word_raw_list.append(word_raw_list_i)
        word_raw_rest_list.append(word_raw_rest_list_i)
    return word_raw_list, word_raw_rest_list

def print_eval_text(feature, i_batch, outf, tokenizer_GPT2, inner_idx_tensor, gen_sent_tensor, gen_sent_tensor_org, result_stats, word_raw_list, word_raw_rest_list, readable_context, run_eval):
    #batch_size, num_head, top_k, n_basis = top_index.size()
    #num_sent_gen = gen_sent_tensor.size(2)
    batch_size, num_head, num_sent_gen, gen_sent_len = gen_sent_tensor.size()
    
    #not_ascii_multi = 0
    #not_ascii_org = 0
    for i_sent in range(batch_size):
        outf.write('batch number: ' + str(i_sent) + '\n')
        last_end = -1
        # for m in range(1):
        for m in range(num_head):
            outf.write('number of head: ' + str(m) + '\n')
            end = inner_idx_tensor[i_sent,m].item()
            if end == last_end:
                continue
            last_end = end
            
            #outf.write(tokenizer_GPT2.convert_tokens_to_string(feature_text[i_sent][:end])+'\n')
            context = tokenizer_GPT2.decode(feature[i_sent,:end])
            
            if readable_context:
                context, bad_context = preprocessing_context(context, outf)
                if bad_context:
                    continue

            outf.write(context+'\n')
            outf.write('\n')
            
            multi_sent_list = []
            org_sent_list = []
            for j in range(num_sent_gen):
#                 print(gen_sent_tensor)
#                 generated_sent = tokenizer_GPT2.decode(gen_sent_tensor[i_sent, m, j, :])
#                 multi_sent_list.append(generated_sent)
                if gen_sent_tensor_org.size(0) > 0:
                    generated_sent_org = tokenizer_GPT2.decode( gen_sent_tensor_org[i_sent, m, j, :] )
                    org_sent_list.append(generated_sent_org)

#             for j in range(num_sent_gen):
#                 generated_sent = multi_sent_list[j]
#                 print_sampled_sent(generated_sent, outf, 'multi-facet '+ str(j))
#                 if run_eval:
#                     result_stats.update("Multi-facet", gen_sent_tensor[i_sent, m, j, :], feature[i_sent,:end], tokenizer_GPT2, word_raw_list[i_sent][m], word_raw_rest_list[i_sent][m], m)
#             if run_eval:
#                 result_stats.update_self_BLEU("Multi-facet", m)
            if gen_sent_tensor_org.size(0) > 0:
                for j in range(num_sent_gen):
                    generated_sent_org = org_sent_list[j]
                    print_sampled_sent(generated_sent_org, outf, 'single-facet '+ str(j))
                    if run_eval:
                        result_stats.update("Single-facet", gen_sent_tensor_org[i_sent, m, j, :], feature[i_sent,:end], tokenizer_GPT2, word_raw_list[i_sent][m], word_raw_rest_list[i_sent][m], m)
                if run_eval:
                    result_stats.update_self_BLEU("Single-facet", m)

            if run_eval:
                result_stats.renew_ngram(m)
    #outf.write('Number of not ascii code in multi: {}, in single: {}: {}\n'.format(not_ascii_conditional, not_ascii_org))

In [11]:
def visualize_interactive_LM(model_multi, model_single, gpt2_model, device, num_sent_gen, gen_sent_len, dataloader, outf, max_batch_num, tokenizer_GPT2, bptt, readable_context = False, run_eval = True):
    top_k = 5
    nlp = English()

    #emb_sum = torch.sum(word_norm_emb,dim=1)
    #OOV_list = torch.nonzero(emb_sum == 0).squeeze().cpu().tolist()
    #print("OOV number = {}".format(len(OOV_list)))
    #print("OOV index examples {}".format(OOV_list[:10]))
    #OOV_set = set(OOV_list)

    with torch.no_grad():
        if run_eval:
            result_stats = result_statistics(gpt2_model)
#             result_stats.add_model("Multi-facet")
            result_stats.add_model("Single-facet")
        else:
            result_stats = []
        for i_batch, sample_batched in enumerate(dataloader):
            sample_batched = sample_batched.to(device)
            # if i_batch == 0:
            #     continue
            print("batch"+str(i_batch))
            sys.stdout.flush()
            
            feature = sample_batched
            
            feature_text = [ [tokenizer_GPT2._convert_id_to_token(x) for x in feature[i,:].tolist()] for i in range(feature.size(0))]

            max_prompt_len = bptt - gen_sent_len
            min_prompt_len = 20
            interval = gen_sent_len
            
            batch_size = feature.size(0)
            start_idx_list = list(range(min_prompt_len,max_prompt_len,interval))
            num_head = len(start_idx_list)
            inner_idx_tensor = torch.tensor(start_idx_list, dtype=torch.long, device = device)
            inner_idx_tensor = inner_idx_tensor.expand(batch_size, num_head)
            batch_size, num_head = inner_idx_tensor.size()

            gen_sent_tensor = torch.empty( (batch_size, num_head, num_sent_gen, gen_sent_len), dtype=torch.long, device=device )
            gen_sent_tensor_org = torch.empty( (batch_size, num_head, num_sent_gen, gen_sent_len), dtype=torch.long, device=device )

            word_raw_list, word_raw_rest_list = get_word_list_spacy(inner_idx_tensor, feature_text, tokenizer_GPT2, nlp)
            # for i_sent in range(1):
            for i_sent in range(batch_size):
                print("sent"+str(i_sent))
                last_end = -1
                
                # for m in range(1):
                for m in range(num_head):
                    print("head"+str(m))

                    end = inner_idx_tensor[i_sent,m]
                    if end == last_end:
                        continue
                    last_end = end
                    
                    context = tokenizer_GPT2.convert_tokens_to_string(feature_text[i_sent][:end])
                    if readable_context:
                        context_proc, bad_context = preprocessing_context(context, outf)
                        if bad_context:
                            continue

                    end_int = end.item()
                    
                    start_int = 0
                    #if end_int > max_prompt_len:
                    #    start_int = end_int - max_prompt_len
                    
#                     t = time.time()
                    feature_expanded = feature[i_sent,start_int:end].unsqueeze(0).expand(num_sent_gen,end_int - start_int).to(device = device)
#                     if model_multi.output_probs:
#                         output = sample_seq_prob(model_multi, feature_expanded, gen_sent_len, device)
#                     else:
#                         output = sample_seq(model_multi, feature_expanded, gen_sent_len, device)
#                     multi_elapsed = time.time() - t
#                     gen_sent_tensor[i_sent, m, :, :] = output
                    
                    t = time.time()
#                     if model_single.output_probs:
#                         output_org = sample_seq_prob(model_single, feature_expanded, gen_sent_len, device)
#                     else:
                    output_org = sample_seq_stage2(model_single, feature_expanded, gen_sent_len, device)
                    org_elapsed = time.time() - t
                    gen_sent_tensor_org[i_sent, m, :, :] = output_org
                    
                    
#                     if run_eval:
#                         #result_stats.model_results["time_count"] += 1
#                         for method_name, time_spent in [ ("Multi-facet",multi_elapsed), ("Single-facet", org_elapsed)]:
#                             result_stats.model_results[method_name]["time_sum"] += time_spent
#                             result_stats.model_results[method_name]["time_count"] += 1

                
            print_eval_text(feature, i_batch, outf, tokenizer_GPT2, inner_idx_tensor, gen_sent_tensor, gen_sent_tensor_org, result_stats, word_raw_list, word_raw_rest_list,  readable_context, run_eval)
            if i_batch + 1 >= max_batch_num:
                break
        if run_eval:
            result_stats.print()
            result_stats.generate_report(outf)

In [12]:
gpt2_org_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
outf = open('generated_stage2.txt', 'w')

In [13]:
t0 = time.time()

In [16]:
visualize_interactive_LM(model_multi=None, model_single=model, gpt2_model=gpt2_org_model, 
                         device=device, num_sent_gen=3, gen_sent_len=30, dataloader=inside_validation_dataloader,
                         outf=outf, max_batch_num=300, tokenizer_GPT2=tokenizer_GPT2, bptt=MAX_LEN, 
                         readable_context = False, run_eval = True)


In [15]:
format_time(time.time() - t0)

'1:23:32'